# Assignment 2
Solution by Wiktor Dobrosierdow (Group 72)

In [1]:
# required imports
from itertools import combinations
from collections import defaultdict
from multiprocessing import Pool, cpu_count
import time
import pandas as pd

## Task 1
First task is the A-Priori algorithm. There are several supporting functions described below.

### Generating candidate itemsets
This function will generate candidate $k$-itemsets from the provided pool of *itemsets*.
The two outer loops will process pairs of subsets from *itemsets*, generating a potential candidate.
The candidate is then checked for validity according to the algorithm rules, and finally added to the list of suitable candidates.

In [2]:
def generate_candidate_itemsets(itemsets, k):
    candidates = set()
    itemsets = list(itemsets)
    for i, A in enumerate(itemsets):
        for B in itemsets[i + 1:]:
            candidate = A | B
            if not (len(candidate) == k and len(A & B) == k - 2):
                continue
            subsets = map(lambda item: candidate - frozenset((item,)), candidate)
            if not all(subset in itemsets for subset in subsets):
                continue
            candidates.add(candidate)
    return candidates

### Pruning
Rather simple, this function filters out itemsets which are below the support threshold.

In [3]:
def prune(freq_itemsets, threshold):
    return {itemset:support for itemset, support in freq_itemsets.items() if support >= threshold}

### Counting singletons
This function starts the A-Priori algorithm by counting the number of $1$-itemsets (singletons).

In [4]:
def freq_singletons(dataset, threshold):
    freq_items = defaultdict(int)
    for row in dataset:
        for item in row:
            freq_items[item] += 1

    return {frozenset((item,)):support for item, support in freq_items.items() if support >= threshold}

### Candidates' support
This function is used in the main loop to count the support of the generated candidate itemsets.
There is an optimization for $k < 3$ (i.e. $k = 2$), where the number of candidates can be very large, and it is more efficient to simply generate all possible combinations of length $k = 2$ for each row, and check whether those combinations are a candidate.
This optimization only works for low $k$, as larger values result in exponentially more combinations.
It does provide a very sizeable peformance improvement for the first pass of $k = 2$ however.

In [5]:
def support_count(dataset, candidates, k):
    freq_itemsets = defaultdict(int)
    if k < 3:
        for row in dataset:
            for subset in map(frozenset, combinations(row, k)):
                if subset in candidates:
                    freq_itemsets[subset] += 1
    else:
        for row in dataset:
            for candidate in candidates:
                if candidate.issubset(row):
                    freq_itemsets[candidate] += 1
    return freq_itemsets

### Multiprocessing support
Splitting the support counting calculation over several cores provides a good boost to performance. It would make sense to only use this when number of candidates is large, but it does not result in too much overhead even with a low number of candidates.

In [6]:
def worker(args):
    return support_count(*args)

def support_count_mp(dataset, candidates, k):
    numproc = cpu_count()
    chunk_size = len(dataset) // numproc
    chunks = [dataset[i:i + chunk_size] for i in range(0, len(dataset), chunk_size)]
    args = [(chunk, candidates, k) for chunk in chunks]

    with Pool(numproc) as pool:
        results = pool.map(worker, args)

    total_freq_itemsets = defaultdict(int)
    for result in results:
        for itemset, support in result.items():
            total_freq_itemsets[itemset] += support

    return total_freq_itemsets

### A-Priori
The star of the show, the `apriori` function. It combines the previously defined helper functions to run the actual algorithm. The frequent itemsets are initialized from the $1$-itemsets, and then progressively extended after testing larger and larget values of $k$.

In [7]:
def apriori(dataset, threshold):
    freq_itemsets = freq_singletons(dataset, threshold)

    last_freq_itemsets = freq_itemsets.copy()

    k = 2
    print(len(dataset))
    while last_freq_itemsets:
        candidates = generate_candidate_itemsets(last_freq_itemsets, k)
        print(f'k = {k}, candidates ({len(candidates)})')

        if not candidates:
            break

        t1 = time.time()
        tmp_freq_itemsets = support_count_mp(dataset, candidates, k)
        print(f'tmp_freq took {time.time() - t1} seconds')

        last_freq_itemsets = prune(tmp_freq_itemsets, threshold)
        freq_itemsets.update(last_freq_itemsets)
        k += 1
    print(f'final k: {k}')

    return freq_itemsets

### Dataset loading
The dataset is a list of transactions separated by newlines. Each transaction is a list of product IDs separated by spaces.

In [8]:
%%time
def load_dataset(path):
    with open(path, 'r') as fp:
        dataset = [set(map(int, line.split())) for line in fp.read().splitlines()]
    return dataset

dataset = load_dataset('T10I4D100K.dat')
print(f'Dataset size: {len(dataset)}')
if 0:
    print('Dataset sample:')
    for row in dataset[:5]:
        print(row)

Dataset size: 100000
CPU times: user 221 ms, sys: 55.5 ms, total: 276 ms
Wall time: 277 ms


In [9]:
%%time
support_threshold = 500
freq_itemsets = apriori(dataset, support_threshold)

100000
k = 2, candidates (161596)
tmp_freq took 3.6372265815734863 seconds
k = 3, candidates (170)
tmp_freq took 1.2574200630187988 seconds
k = 4, candidates (43)
tmp_freq took 0.28475117683410645 seconds
k = 5, candidates (10)
tmp_freq took 0.2430574893951416 seconds
k = 6, candidates (1)
tmp_freq took 0.22603464126586914 seconds
final k: 7
CPU times: user 5.19 s, sys: 1.28 s, total: 6.47 s
Wall time: 7.49 s


In [10]:
frame = pd.DataFrame([
    {'Itemset': itemset, 'Support': support}
    for itemset, support in freq_itemsets.items()
]).sort_values('Support', ascending=False)
frame2 = pd.DataFrame([
    {'support': int(support), 'itemsets': itemsets}
    for itemsets, support in freq_itemsets.items()
])
#del freq_itemsets

print('Frequent Itemsets and their Support:')
display(frame)
del frame

Frequent Itemsets and their Support:


,Itemset,Support
7,(368),7828
62,(529),7057
223,(829),6810
49,(766),6265
164,(722),5845
...,...,...
799,"(75, 325)",500
598,"(569, 461)",500
791,"(120, 638)",500
948,"(192, 935, 487)",500


## Bonus Task
The task is to use the frequent itemsets generated from the A-Priori algorithm to generate association rules between them.

In [11]:
def generate_association_rules(freq_itemsets, min_confidence):
    rules = []
    for itemset, support in freq_itemsets.items():
        if len(itemset) < 2:
            # cannot generate any rules from 1-itemsets
            continue

        # generate all possible non-empty proper subsets of the itemset
        subsets = (
            frozenset(subset)
            for i in range(1, len(itemset))
            for subset in combinations(itemset, i)
        )
        for subset in subsets:
            remainder = itemset - subset
            if len(remainder) == 0:
                continue

            # calculate the confidence of the rule subset -> remainder
            subset_support = freq_itemsets.get(subset, 0)
            confidence = support / subset_support if subset_support > 0 else 0

            if abs(confidence) >= min_confidence:
                rules.append((subset, remainder, support, confidence))
    return rules

# Define minimum confidence threshold
min_confidence = 0.75

# Generate rules from the frequent itemsets
association_rules = generate_association_rules(freq_itemsets, min_confidence)
df = pd.DataFrame(association_rules, columns=['Antecedent', 'Consequent', 'Support', 'Confidence'])
df.sort_values(by='Confidence', ascending=False)
display(df)

,Antecedent,Consequent,Support,Confidence
0,(626),(496),761,0.870709
1,(801),(392),664,0.795210
2,(801),(862),674,0.807186
3,(842),(579),600,0.797872
4,(842),(803),607,0.807181
...,...,...,...,...
959,"(217, 546, 947, 661)",(923),559,0.977273
960,"(546, 947, 923, 661)",(217),559,0.967128
961,"(217, 546, 947, 923)",(661),559,0.991135
962,"(217, 546, 923, 661)",(947),559,0.978984


The cells below were used to check the algorithm against a library-implemented one, but are no longer necessary. They are included in case confirmation is needed that the algorithm implemented above is indeed correct.

In [12]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

from mlxtend.frequent_patterns import apriori

In [13]:
frame3['support'] *= len(dataset)
frame3['support'] = frame3['support'].apply(lambda x: int(round(x)))
frame3.sort_values('support', ascending=False)
display(frame3)

NameError: name 'frame3' is not defined

In [ ]:
merged2 = frame3.merge(frame2, how='outer', indicator=True)
merged2[merged2['_merge'] != 'both'].sort_values('support')